In [1]:
import open3d as o3d
import matplotlib.pyplot as plt
import numpy as np

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


点云文件读取
读取TXT文件，转化为ply文件，有关文件格式的信息可以看中文文档

In [17]:
# 读取点云

print("->正在加载点云... ")
# 记得改路径
pcd = o3d.io.read_point_cloud("F:/1Bo7_vs/basic_data/input.txt", format='xyz')

print(pcd)

print("->正在读取点云")
o3d.io.write_point_cloud("transformed.ply", pcd)	
print(pcd)

->正在加载点云... 
PointCloud with 11769609 points.
->正在读取点云
PointCloud with 11769609 points.


点云可视化
数据大比较慢，请耐心等待(待优化)

In [ ]:

print("->正在加载点云... ")
pcd = o3d.io.read_point_cloud("transformed.ply")
print(pcd)

print("->正在可视化点云")
o3d.visualization.draw_geometries([pcd])


->正在加载点云... 
PointCloud with 11769609 points.
->正在可视化点云


点云的体素下采样
Open3D 的函数失效了。。。实际上不能降采样

In [19]:
import open3d as o3d
import numpy as np

print("->正在加载点云... ")
pcd = o3d.io.read_point_cloud("trans.pcd")
print(pcd)

print("->正在可视化原始点云")
o3d.visualization.draw_geometries([pcd])

print("->正在体素下采样...")
voxel_size = 0.5
downpcd = pcd.voxel_down_sample(voxel_size)
print(downpcd)

print("->正在可视化下采样点云")
o3d.visualization.draw_geometries([downpcd])


->正在加载点云... 
PointCloud with 11769609 points.
->正在可视化原始点云
->正在体素下采样...
PointCloud with 11769609 points.
->正在可视化下采样点云


统计滤波

点云数据中存在大量噪声点，这些噪声点会影响后续的分析结果。因此，需要对点云数据进行统计滤波，去除噪声点。

radius_outlier_removal 移除给定球体中几乎没有邻居的点。需要两个参数：

num_points，邻域球内的最少点数，低于该值的点为噪声点

radius ，邻域半径的大小

In [24]:


print("->正在加载点云... ")
pcd = o3d.io.read_point_cloud("trans.pcd")
print("原始点云：", pcd)

# ------------------------- 统计滤波 --------------------------
print("->正在进行统计滤波...")
num_neighbors = 20 # K邻域点的个数
std_ratio = 0.5 # 标准差乘数
# 执行统计滤波，返回滤波后的点云sor_pcd和对应的索引ind
sor_pcd, ind = pcd.remove_statistical_outlier(num_neighbors, std_ratio)
sor_pcd.paint_uniform_color([0, 0, 1])
print("统计滤波后的点云：", sor_pcd)

#RGB,此处B(ule)值为1，显示为蓝色，噪声点云为红色

sor_pcd.paint_uniform_color([0, 0, 1])
# 提取噪声点云
sor_noise_pcd = pcd.select_by_index(ind,invert = True)
print("噪声点云：", sor_noise_pcd)
sor_noise_pcd.paint_uniform_color([1, 0, 0])
# ===========================================================

# 可视化统计滤波后的点云和噪声点云
o3d.visualization.draw_geometries([sor_pcd, sor_noise_pcd])


->正在加载点云... 
原始点云： PointCloud with 11769609 points.
->正在进行统计滤波...
统计滤波后的点云： PointCloud with 10820836 points.
噪声点云： PointCloud with 948773 points.


点云法线估计(计算坡度的重要依据)

In [25]:
print("->正在加载点云... ")
pcd = o3d.io.read_point_cloud("trans.pcd")
print(pcd)

print("->正在估计法线并可视化...")
radius = 0.01   # 搜索半径
max_nn = 20     # 邻域内用于估算法线的最大点数
pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius, max_nn))     # 执行法线估计
o3d.visualization.draw_geometries([pcd], point_show_normal=True)

print("->正在打印前10个点的法向量...")
print(np.asarray(pcd.normals)[:10, :])

->正在加载点云... 
PointCloud with 11769609 points.
->正在估计法线并可视化...
->正在打印前10个点的法向量...
[[0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]]


点云分割
DBSCAN 聚类分割
Open3D实现了DBSCAN[1996]，这是一种基于密度的聚类算法。该算法在cluster_dbscan中实现，需要两个参数：eps 为同一簇内的最大点间距，min_points 定义有效聚类的最小点数。函数返回标签 label，其中label = -1表示噪声。

注：现在还未找到合适的eps和min_points参数，聚类效果差，等待后续工作。

In [31]:
print("->正在加载点云... ")
pcd = o3d.io.read_point_cloud("trans.pcd")
print(pcd)

print("->正在DBSCAN聚类...")
eps = 15           # 同一聚类中最大点间距
min_points = 10     # 有效聚类的最小点数
with o3d.utility.VerbosityContextManager(o3d.utility.VerbosityLevel.Debug) as cm:
    labels = np.array(pcd.cluster_dbscan(eps, min_points, print_progress=True))
max_label = labels.max()    # 获取聚类标签的最大值 [-1,0,1,2,...,max_label]，label = -1 为噪声，因此总聚类个数为 max_label + 1
print(f"point cloud has {max_label + 1} clusters")
colors = plt.get_cmap("tab20")(labels / (max_label if max_label > 0 else 1))
colors[labels < 0] = 0  # labels = -1 的簇为噪声，以黑色显示
pcd.colors = o3d.utility.Vector3dVector(colors[:, :3])
o3d.visualization.draw_geometries([pcd])

->正在加载点云... 
PointCloud with 11769609 points.
->正在DBSCAN聚类...
[Open3D DEBUG] Precompute neighbors.
[Open3D DEBUG] Done Precompute neighbors.
[Open3D DEBUG] Compute Clusters
[Open3D DEBUG] Done Compute Clusters: 1
point cloud has 1 clusters


RANSAC 平面分割
蓝色为平面

注：未找到最佳的distance_threshold，可能需要分块处理加多次尝试，不可能对原始点云数据整体拟合一个平面

In [34]:
print("->正在加载点云... ")
pcd = o3d.io.read_point_cloud("trans.pcd")
print(pcd)

print("->正在RANSAC平面分割...")
distance_threshold = 4    # 内点到平面模型的最大距离
ransac_n = 3                # 用于拟合平面的采样点数
num_iterations = 1000       # 最大迭代次数

# 返回模型系数plane_model和内点索引inliers，并赋值
plane_model, inliers = pcd.segment_plane(distance_threshold, ransac_n, num_iterations)

# 输出平面方程
[a, b, c, d] = plane_model
print(f"Plane equation: {a:.2f}x + {b:.2f}y + {c:.2f}z + {d:.2f} = 0")

# 平面内点点云
inlier_cloud = pcd.select_by_index(inliers)
inlier_cloud.paint_uniform_color([0, 0, 1.0])
print(inlier_cloud)

# 平面外点点云
outlier_cloud = pcd.select_by_index(inliers, invert=True)
outlier_cloud.paint_uniform_color([1.0, 0, 0])
print(outlier_cloud)

# 可视化平面分割结果
o3d.visualization.draw_geometries([inlier_cloud, outlier_cloud])

->正在加载点云... 
PointCloud with 11769609 points.
->正在RANSAC平面分割...
Plane equation: -0.02x + 0.02y + 1.00z + -6767.69 = 0
PointCloud with 432067 points.
PointCloud with 11337542 points.


点云曲面重建
在许多情况下，我们希望生成密集的三维几何体，即三角形网格。然而，从多视图立体方法或深度传感器中，我们只能获得非结构化点云。要从非结构化输入中获得三角形网格，我们需要执行曲面重建。文献中有几种方法，Open3D目前实现了以下功能：

Alpha shapes [Edelsbrunner1983]
Ball pivoting [Bernardini1999]
Poisson surface reconstruction [Kazhdan2006]

Alpha shapes
Alpha shapes 是凸壳的推广。正如这里所描述的，我们可以直观地认为Alpha shapes如下：想象一个巨大的冰淇淋，其中包含点S作为硬巧克力块。使用其中一个球形冰淇淋勺，我们可以在不撞到巧克力块的情况下雕刻出冰淇淋块的所有部分，从而甚至在内部雕刻出孔（例如，仅从外部移动勺子无法触及的部分）。我们最终将得到一个（不一定是凸的）对象，该对象以帽、弧和点为边界。如果我们现在把所有的面拉直成三角形和线段，我们就可以直观地描述S的Alpha shapes。

Open3D实现了 create_from_point_cloud_alpha_shape 方法

可调参数：alpha：控制凸壳的大小，值越大，凸壳越大。
直观来说值越小越接近点云数据的形状
消耗的计算资源大，可能导致内核崩溃

In [2]:

# --------------------------- 加载点云 ---------------------------
print("->正在加载点云... ")
pcd = o3d.io.read_point_cloud("trans.pcd")
print("原始点云：", pcd)
# ==============================================================

# ------------------------- Alpha shapes -----------------------
alpha = 0.06
print(f"alpha={alpha:.3f}")
mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_alpha_shape(pcd, alpha)
mesh.compute_vertex_normals()
o3d.visualization.draw_geometries([mesh], mesh_show_back_face=True)
# ==============================================================


->正在加载点云... 


Ball pivoting
Ball pivoting（BPA）[Bernardini1999]是一种与Alpha shapes相关的曲面重建方法。直观地说，想象一个具有给定半径的3D球，我们将其落在点云上。如果它击中任何3个点（并且没有穿过这3个点），它将创建一个三角形。然后，该算法从现有三角形的边开始旋转，每当它击中球未落下的3个点时，我们创建另一个三角形。

open3D 对应的函数为 create_from_point_cloud_ball_pivoting

读取ply文件的信息(transformed.ply中只包含顶点数据，不包含网格数据)
接下来我们将创建网格数据

In [3]:
# 读取PLY文件
mesh = o3d.io.read_triangle_mesh("transformed.ply")

# 打印三角网格的顶点数和面数
print("顶点数:", len(mesh.vertices))
print("面数:", len(mesh.triangles))

[Open3D WARNING] geometry::TriangleMesh appears to be a geometry::PointCloud (only contains vertices, but no triangles).
顶点数: 11769609
面数: 0


使用numpy数组读取点云的坐标数据

In [4]:
import open3d as o3d

# 读取PLY文件
pcd = o3d.io.read_point_cloud("transformed.ply")

# 获取顶点数据
vertices = np.asarray(pcd.points)

# 打印顶点数据的形状和前几个顶点
print("顶点数据形状:", vertices.shape)
print("前五个顶点:", vertices[:5])


顶点数据形状: (11769609, 3)
前五个顶点: [[ 69263.7 144566.7   5304.8]
 [ 69263.7 144562.2   5304. ]
 [ 69263.7 144557.7   5303.2]
 [ 69263.7 144553.2   5302.4]
 [ 69263.7 144548.7   5301.8]]


Poisson surface reconstruction [Kazhdan2006]
根据点云数据，创造三角网格数据，并保存为mesh_with_triangles.ply

泊松曲面重建方法[Kazhdan2006]解决了一个正则化优化问题，以获得光滑曲面。因此，泊松曲面重建比上述方法更可取，因为它们会产生非平滑结果，因为点云的点也是生成的三角形网格的顶点，无需任何修改。

Open3D对应的方法为 create_from_point_cloud_poisson，这基本上是Kazhdan代码的包装。该函数的一个重要参数是depth，它定义了用于曲面重建的八叉树的深度，因此表示生成的三角形网格的分辨率。depth值越高，网格的细节就越多。

In [11]:

# 读取PLY文件
pcd = o3d.io.read_point_cloud("transformed.ply")

# 估计点云的法线
pcd.estimate_normals()

# 使用点云数据创建三角网格
mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(pcd, depth=9)

# 可选：平滑网格以提高三角网格质量

mesh.filter_smooth_taubin(number_of_iterations=1000, lambda_filter=0.5, mu=-0.53)

# 保存三角网格到文件
o3d.io.write_triangle_mesh("mesh_with_triangles.ply", mesh)


True

In [14]:
#一些可视化
print(mesh)
#设置为蓝色
mesh.paint_uniform_color([0, 0, 1])
o3d.visualization.draw_geometries([mesh])

TriangleMesh with 231689 points and 463128 triangles.


In [28]:
o3d.visualization.draw_geometries([pcd])

In [27]:
#尝试降采样依旧失败。。。。
print("->正在体素下采样...")
voxel_size = 0.5
downpcd = pcd.voxel_down_sample(voxel_size)
print(downpcd)

print("->正在可视化下采样点云")
print(downpcd)
o3d.visualization.draw_geometries([downpcd])

->正在体素下采样...
PointCloud with 9 points.
->正在可视化下采样点云
PointCloud with 9 points.


mesh 方法

In [29]:


# -------------------------- 定义点云体素化函数 ---------------------------
def get_mesh(_relative_path):
    mesh = o3d.io.read_triangle_mesh(_relative_path)
    mesh.compute_vertex_normals()
    return mesh
# =====================================================================

# -------------------- Poisson surface reconstruction ------------------
# 加载点云
print("->Poisson surface reconstruction...")
_relative_path = "mesh_with_triangles.ply"    # 设置相对路径
N = 10000                        # 将点划分为N个体素
pcd = get_mesh(_relative_path).sample_points_poisson_disk(N)
pcd.normals = o3d.utility.Vector3dVector(np.zeros((1, 3)))  # 使现有法线无效

# 法线估计
pcd.estimate_normals()
o3d.visualization.draw_geometries([pcd], point_show_normal=True)
pcd.orient_normals_consistent_tangent_plane(100)
o3d.visualization.draw_geometries([pcd], point_show_normal=True)

# 泊松重建
print('run Poisson surface reconstruction')
with o3d.utility.VerbosityContextManager(
        o3d.utility.VerbosityLevel.Debug) as cm:
    mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(
        pcd, depth=9)
print(mesh)
mesh.paint_uniform_color([0, 0, 1])
o3d.visualization.draw_geometries([mesh])
# =====================================================================


->Poisson surface reconstruction...
run Poisson surface reconstruction
[Open3D DEBUG] Input Points / Samples: 10000 / 10000
[Open3D DEBUG] #   Got kernel density: 0.029 (s), 325.41 (MB) / 1603.63 (MB) / 3963 (MB)
[Open3D DEBUG] #     Got normal field: 0.017 (s), 325.797 (MB) / 1603.63 (MB) / 3963 (MB)
[Open3D DEBUG] Point weight / Estimated Area: 1.395311e-04 / 1.395311e+00
[Open3D DEBUG] #       Finalized tree: 0.049 (s), 331.566 (MB) / 1603.63 (MB) / 3963 (MB)
[Open3D DEBUG] #  Set FEM constraints: 0.0640001 (s), 329.055 (MB) / 1603.63 (MB) / 3963 (MB)
[Open3D DEBUG] #Set point constraints: 0.0319998 (s), 329.09 (MB) / 1603.63 (MB) / 3963 (MB)
[Open3D DEBUG] Leaf Nodes / Active Nodes / Ghost Nodes: 280435 / 164088 / 156409
[Open3D DEBUG] Memory Usage: 329.090 MB
[Open3D DEBUG] # Linear system solved: 0.889 (s), 349.812 (MB) / 1603.63 (MB) / 3963 (MB)
[Open3D DEBUG] Got average: 0.00500011 (s), 337.254 (MB) / 1603.63 (MB) / 3963 (MB)
[Open3D DEBUG] Iso-Value: 5.158620e-01 = 5.158621e+

In [30]:
mesh.paint_uniform_color([0, 0, 1])
o3d.visualization.draw_geometries([mesh])

点云的体素化

In [32]:

# ---------------------- 定义点云体素化函数 ----------------------
def get_mesh(_relative_path):
    mesh = o3d.io.read_triangle_mesh(_relative_path)
    mesh.compute_vertex_normals()
    return mesh
# =============================================================

# ------------------------- 点云体素化 --------------------------
print("->正在进行点云体素化...")
_relative_path = "mesh_with_triangles.ply"    # 设置相对路径
N = 10000        # 将点划分为N个体素
pcd = get_mesh(_relative_path).sample_points_poisson_disk(N)

# fit to unit cube
pcd.scale(1 / np.max(pcd.get_max_bound() - pcd.get_min_bound()),
          center=pcd.get_center())
pcd.colors = o3d.utility.Vector3dVector(np.random.uniform(0, 1, size=(N, 3)))
print("体素下采样点云：", pcd)
print("正在可视化体素下采样点云...")
o3d.visualization.draw_geometries([pcd])

print('执行体素化点云')
voxel_grid = o3d.geometry.VoxelGrid.create_from_point_cloud(pcd, voxel_size=0.05)
print("正在可视化体素...")
o3d.visualization.draw_geometries([voxel_grid])
# ===========================================================


->正在进行点云体素化...
体素下采样点云： PointCloud with 10000 points.
正在可视化体素下采样点云...
执行体素化点云
正在可视化体素...
